# Offside

## Introduction, Reference Domain, and Objectives
The Offside project starts from the need of creating a recommendation system based on machine learning which, given a football player, returns a series of players as similar as possible based on a set of technical-tactical data. The reference domain is, obviously, the world of football, and the reference platform for data retrieval is Fbref, a consultation source that includes data relating to players, teams and championships in an easily to consult, -and scrape-, fashion.

The approach that the project intends to adopt is based on a rather simple notion: let's assume we have a two-dimensional Cartesian plane, where each point corresponds to a player based on a pair of technical characteristics of our choice. The players most similar to the original player will be those who have the smallest distance on the Cartesian plane. It is sufficient to raise the plane to n dimensions, one for each of the technical attributes considered, to evaluate the similarity of the players on the hyperplane. The only caveat is that calculating the distance on a n-dimensional plane requires a little bit more effort than doing the same thing for a bi-dimensional plane, but thanks to the use of the Euclidean distance, the problem will be solved quite easily. 

## Parametric Configuration

The following block of code is dedicated to the configuration of the model, which was designed to be as modular as possible in order to be reused according to the individual needs of sports data analysis professionals. In particular, the code first allows us to specify the name and surname of the player we intend to take as a reference. Simply pass a string to the code containing the name of the player contained in the Column `(Background Information, Full Name)`, for example `reference_player_name = "Olivier Giroud"`. Obviously, the player must be present in the Table, otherwise an error will be thrown. We then have a role filter, called `role_filter`. If this variable is set to `None`, then no filter will be used and the search will take place taking the players' statistical differences as a guideline. If, however, we have a specific position in mind for which to scout a player, for example the goalkeeper, just write `role_filter = "GK"`, and all the players returned will be players with `"GK"` as the value of their `(Background Information, Position)` column. This can be particularly useful when the starting point of the search is a goalkeeper with good feet (i.e. Mike Maignan), who due to his high ability to generate assists, without the filter activated could be compared with defenders or midfielders

We then have the variable `number_of_players_returned`, which indicates the quantity of players that will be returned by the algorithm. By default, this parameter is set to 5, but you can change it to have a longer list of similarities. It goes without saying that the longer the list, the less similar the values at the bottom of the list are to the original player. We then have the `relevant_cols` Array, which contains a list chosen by the analyst of the attributes to be included in the Euclidean comparison of the players. The columns to be inserted must have the same name as the corresponding columns on the dataframe, paying careful attention to Case Sensitivity: uppercase and lowercase are treated differently, and this could cause errors. By default, the vast majority of the columns present have been selected, as this allows for an all-round analysis of the players and their respective comparisons between them. However, it is possible to remove or add values (simply by commenting the respective line) based on scouting needs. Finally, the variable `league_filter` allows the user to specify a list of leagues to be considered in the search. This filter ensures that only players from the specified leagues are included in the comparison. For example, setting `league_filter = ["Serie A", "Premier League", "Bundesliga", "La Liga", "Ligue 1", "Eredivisie"]` restricts the search to players in these top European leagues. 

In [3]:
"""
-* MODEL CONFIGURATION *-
The following block of code contains the configuration parameters of the model, designed for modularity in sports data analysis. 
It enables customization according to individual needs, so please follow this explanation of the editable values in order to make the best out of it.

1. reference_player_name: identifies the player to be brought up for comparison.
   Please provide the full name as listed in the dataset under the (Background Information, Full Name) column.

2. role_filter: specifies the positional filter. 
   If set to None, the algorithm compares players based on statistical differences. 
   If set to a specific role abbreviation, the search is narrowed down to players with the same position.
   Useful for focused searches, ensuring relevant comparisons.
   Definitions of player roles:
   None - No Filter Active 
   "GK" - Goalkeepers
   "DF" - Defenders
   "MF" - Midfielders
   "FW" - Forwards
   "FB" - Fullbacks
   "LB" - Left Backs
   "RB" - Right Backs
   "CB" - Center Backs
   "DM" - Defensive Midfielders
   "CM" - Central Midfielders
   "LM" - Left Midfielders
   "RM" - Right Midfielders
   "WM" - Wide Midfielders
   "LW" - Left Wingers
   "RW" - Right Wingers
   "AM" - Attacking Midfielders

3. number_of_players_returned: indicates the quantity of players returned by the algorithm.
   Default is set to 5 for concise output, but can be adjusted for more comprehensive lists.
   Note: Increasing the number may reduce similarity among listed players along the bottom of the list.
   
4. relevant_cols: contains the columns of the dataframe that are to be taken in consideration by the algorithm.
   A sensible default value has already been set, but can be adjusted if other scouting needs need to be met.
   
5. league_filter: this parameter is designed to refine player comparisons based on their performance in specific leagues.
    It allows users to specify one or more leagues from which player data will be considered for analysis.
    By setting league_filter to a list of league abbreviations, users can narrow down the comparison to players who have 
    participated in the specified leagues. This ensures that the analysis is tailored to the context of particular 
    football competitions, facilitating more relevant player assessments.
    Supported Leagues:
    "Serie A" 
    "Premier League" 
    "Bundesliga" 
    "La Liga" 
    "Ligue 1"
    "Eredivisie" 
"""

# Set your desired reference player's name
reference_player_name = "Olivier Giroud"

# Set your desired role filter
role_filter = None

# Set your desired number of players to be returned 
number_of_players_returned = 5

# Set your desired relevant columns
relevant_cols = ["(Background Information, Position)", 
               "(Background Information, Year of Birth)",
               "(Playing Time, MP)",
               "(Playing Time, Starts)",
               "(Playing Time, Min)",
               "(Playing Time, 90s)",
               "(Performance, Gls)",
               "(Performance, Ast)",
               "(Performance, G+A)",
               "(Performance, G-PK)",
               "(Performance, PK)",
               "(Performance, PKatt)",
               "(Performance, CrdY)",
               "(Performance, CrdR)",
               "(Expected, xG)",
               "(Expected, npxG)",
               "(Expected, xAG)",
               "(Expected, npxG+xAG)",
               "(Progression, PrgC)",
               "(Progression, PrgP)",
               "(Progression, PrgR)",
               "(Per 90 Minutes, Gls)",
               "(Per 90 Minutes, Ast)",
               "(Per 90 Minutes, G+A)",
               "(Per 90 Minutes, G-PK)",
               "(Per 90 Minutes, G+A-PK)",
               "(Per 90 Minutes, xG)",
               "(Per 90 Minutes, xAG)",
               "(Per 90 Minutes, xG+xAG)",
               "(Per 90 Minutes, npxG)",
               "(Per 90 Minutes, npxG+xAG)"]

# Set your desired league filter
league_filter = ["Serie A", 
                 "Premier League", 
                 "Bundesliga", 
                 "La Liga", 
                 "Ligue 1", 
                 "Eredivisie"
                 ]

data_source = "../Exports/Data/merged.csv"

## Algorithmic Definition and Data Modelling

After the data preparation phase, we can finally enter the modelling phase. The data pre-processing begins with the removal of any rows containing missing values (`NaN`) from the DataFrame containing the football player data, as this will avoid potential errors down the line. Relevant columns (`relevant_cols`) are then selected in order to describe various aspects of the player such as background information, playing time, performance metrics, expected goals, progression, and statistics per 90 minutes. In order to facilitate the normalization process, only the numeric columns are retained from the array of containing the relevant columns. The data is then normalized for each column, in order to create a fairer distribution and result in the evaluation phase.

The real insight behind the construction of the Model is the idea of mapping football players in an n-dimensional plane, based on their statistics and technical characteristics. To do this, we use the Euclidean distance formula, which is applicable to vectors of any size. Instead of implementing it manually, we will use the SciPy `euclidean` Function (https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.euclidean.html), which we have already imported previously with `from scipy .spatial.distance import euclidean`. This formula can be used to calculate the distance between two points in n-dimensional space, and in our example it fits perfectly to the size of the vectors involved. So, whether the vectors are two-dimensional, three-dimensional or higher dimensional, the formula remains the same, and this allows us to reason at an n-dimensional level as we would have done at a two-dimensional level: neighboring points in space equal players with similar characteristics. For completeness, here is the Euclidean Formula used, where `u` and `v` are two vectors of dimension `n`

$$\text{euclidean}(\mathbf{u}, \mathbf{v}) = \sqrt{\sum_{i=1}^{n} (u_i - v_i)^2}$$

The code will compute the Euclidean distances between the provided reference player and all other players in the dataset, filtering by role if specified. The distances are stored in a dictionary where the keys represent player indices and the values represent their respective distances from the reference player. These distances are then sorted in ascending order. The players most similar to the reference player are extracted based on their ascending Euclidean distances, defined as `Delta` in the printed summary, to represent a numerical indicator of distance between the reference player and another player (the lower the delta value, the better the similarity). Finally, the results are printed, formatted for having good readability and insights. If a role filter is applied, a note is provided regarding its influence on the returned results, to avoid potential oversights. It is also good to mention the speed of execution. The Euclidean Distance using the following code can be calculated in `O(n)` time, where `n` is the dimension of the input vectors. This is because the calculation involves only sums, multiplications, and square roots, which can be performed in linear time with respect to the size of the vectors. Therefore, we can also decide to have a huge list of players returned to us, and everything will be calculated in a matter of milliseconds.

In [5]:
import os
import random
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean

#Function to print both to console and file with correct formatting
def print_and_write(file, text):
    print(text)
    file.write(text + "\n")

df = pd.read_csv(data_source)

# Dropping rows with missing values to prevent errors
df.dropna(inplace=True)

# Selecting numeric columns
numeric_cols = df[relevant_cols].select_dtypes(include=['number'])

# Normalizing the data
normalized_df = (numeric_cols - numeric_cols.mean()) / numeric_cols.std()

# Determining the index of the row corresponding to the reference player
reference_player_index = df.index[df['(Background Information, Full Name)'] == reference_player_name][0]

# Calculating distances between the reference player and others
distances = {}
reference_player = normalized_df.loc[reference_player_index]

for i, player in normalized_df.iterrows():
    if i == reference_player_index:
        continue
    # Adding role filtering condition
    if role_filter is not None and df.loc[i]['(Background Information, Position)'] != role_filter:
        continue
    # Adding league filtering condition
    if df.loc[i]['(Background Information, League)'] not in league_filter:
        continue
    distances[i] = euclidean(reference_player, player)

# Sorting distances
sorted_distances = sorted(distances.items(), key=lambda x: x[1])

# Selecting top similar players
top_similar_players = sorted_distances[:number_of_players_returned]

# Create a directory containing the reference player name
os.makedirs(f"Exports/Reports/Similarity/{reference_player_name}", exist_ok=True)

# Saving output to file
with open(f"Exports/Reports/Similarity/{reference_player_name}/{reference_player_name} - Report.txt", "w") as report_file:
    # Printing header to both terminal and file
    header = ("The reference player is {}\n"
              "Below is a list of the players most similar to the player mentioned above,\n"
              "ordered from most to least similar via Euclidean distance sampling.\n\n".format(reference_player_name))
    print_and_write(report_file, header)

    # Printing role filter information to both terminal and file
    if role_filter is not None:
        role_info = ("Please note that the values returned by the model are filtered for the '{}' role.\n"
                     "You can change or remove the filter entirely in the algorithm configuration section at any time.\n\n".format(role_filter))
    else:
        role_info = ("Please note that the values returned by the model are not filtered by any role.\n"
                     "You can add or change the role filter in the algorithm configuration section at any time.\n\n")
    print_and_write(report_file, role_info)

    # Printing league filter information to both terminal and file
    if league_filter:
        league_info = ("The league filter is set to display the following teams: {}\n"
                       "You can also change this setting at any time in the Model Configuration Section.\n\n".format(", ".join(league_filter)))
    else:
        league_info = ("The league filter is not set. You can set it in the Model Configuration Section.\n\n")
    print_and_write(report_file, league_info)

    print('─' * 25 + "\n")
    report_file.write('─' * 25 + "\n\n")

    # Printing details of top similar players to both terminal and file
    index_number = 1
    for player_index, distance in top_similar_players:
        player_name = df.loc[player_index]['(Background Information, Full Name)']
        player_role = df.loc[player_index]['(Background Information, Position)']
        player_club = df.loc[player_index]['(Background Information, Squad)']
        player_age = df.loc[player_index]['(Background Information, Year of Birth)']

        player_info = ("Player #{}: {}\n"
                       "Role: {}\n"
                       "Team: {}\n"
                       "Born In: {}\n"
                       "Delta: {:.5f}\n\n".format(index_number, player_name, player_role, player_club, int(player_age), distance))

        print_and_write(report_file, player_info)

        index_number += 1
    
# Printing a completion message indicating successful report saving
print(f"Report generation completed. The report has been successfully saved to 'Exports/Reports/Similarity/{reference_player_name}/{reference_player_name} - Report.txt'.")

The reference player is Olivier Giroud
Below is a list of the players most similar to the player mentioned above,
ordered from most to least similar via Euclidean distance sampling.


Please note that the values returned by the model are not filtered by any role.
You can add or change the role filter in the algorithm configuration section at any time.


The league filter is set to display the following teams: Serie A, Premier League, Bundesliga, La Liga, Ligue 1, Eredivisie
You can also change this setting at any time in the Model Configuration Section.


─────────────────────────

Player #1: Niclas Füllkrug
Role: FW
Team: Dortmund
Born In: 1993
Delta: 4.75248


Player #2: Antoine Griezmann
Role: FW,MF
Team: Atlético Madrid
Born In: 1991
Delta: 4.81723


Player #3: Victor Osimhen
Role: FW
Team: Napoli
Born In: 1998
Delta: 4.96227


Player #4: Tobias Lauritsen
Role: FW
Team: Sparta R'dam
Born In: 1997
Delta: 5.13983


Player #5: Tobias Lauritsen
Role: FW
Team: Sparta R'dam
Born In: 1997

## Data Visualisation and Plotting


To better understand the results obtained, we will exploit two different plotting techniques, the first will be a classic Horizontal Bar Plot, while the second will be a Radar Plot. Starting with the Bar Plot contained in the following code, this rather simple and intuitive to understand graph shows the results of the distance of the `n` players most similar to the player taken as reference. The x-axis of the graph shows the Delta, i.e. the distance coefficient between the reference player and the players most similar to him. The lower the value, the better the similarity between the two. The y-axis shows the name of the players, ordered from most to least similar to the player who gives the title to the table (in other words, the players are presented in ascending order based on the value of Delta). The color of the bars corresponding to the players displayed in the graph will be useful to visualize at a glance the team to which they belong, as shown in the legend. The primary purpose of this graph is to give a quick and shareable visualization of the result obtained by the algorithm, useful as a clarifying example for non-technical personnel who may need to use the information produced. However, this is not the most in-depth way of analyzing the differences, which is why it will not be the only view adopted.

In [ ]:
# Extracting player names and distances from the DataFrame
similar_player_names = [df.loc[player_index]['(Background Information, Full Name)'] for player_index, _ in top_similar_players]
similar_player_distances = [distance for _, distance in top_similar_players]

# Assigning random colors to players for visual distinction
player_colors = [f'#{random.randint(0, 0xFFFFFF):06x}' for _ in similar_player_names]

# Plotting the horizontal bar chart
plt.figure(figsize=(10, 6))
bars = plt.barh(similar_player_names, similar_player_distances, color=player_colors)

# Labeling axes and title
plt.xlabel('Delta')
plt.ylabel('Player')
plt.title(f'Top Similar Players to Reference Player {reference_player_name} (Lower is Better)')

# Inverting y-axis to display players with the highest similarity at the top
plt.gca().invert_yaxis()  

# Saving the plot as an image file
plt.savefig(f'Exports/Reports/Similarity/{reference_player_name}/{reference_player_name} - Delta Graph.jpg', bbox_inches='tight')

# Displaying the plot
plt.show()

The real complication arising from plotting data beyond three dimensions comes from finding a compromise between making the data as comprehensible as possible, without at the same time giving up the depth and insights they offer. After an analysis of the graphs that can be created with Python (an excellent reference material in this regard is the Python Graph Gallery, that can be found at this URL: https://python-graph-gallery.com), it was clear that the best choice for the problem at hand was the Radar Plot, also known as Spider Plot or Polar Chart. The Plot contained in the following code block was created using MatPlotLib (the documentation used to build the next code block can be found on https://matplotlib.org/stable/gallery/specialty_plots/radar_chart.html, and scattered around StackOverflow Threads and Online Forums), through the creation of the `radar_plot` function. This function takes as input several parameters `player_names`, `player_data`, `reference_data`, `attribute_labels`, and `reference_player_name`. Using the values contained in them, it constructs an informative display, complete with legend. This is by far the most useful graph, as it allows you to observe multivariate data in a clear and efficient way, allowing easy comparison of players based on their attributes. 

The way the Calculation Model is structured generates an interesting graphical phenomenon. As long as you keep the list of suggested players within responsible limits, the plotting creates a clear silhouette of their areas of excellence, -or lack-. Since the model calculates the delta via Euclidean distance, the resulting plots are denoted by a similar shape and distribution within the radar space. Furthermore, such a visualization allows you to focus at a glance on attributes that may be more interesting than others. For example, you can focus on the number of assists you expect, find the player who has the most playing time, or even find the player most capable of scoring goals; and in the list of proposed players find the one who is considered best in that aspect. In this way, the graph not only highlights the actual similarity of the players compared to the reference player, but also proves to be a valuable tool in moments when it is necessary to make an informed decision based on the available data.

In [ ]:
# Removing irrelevant columns from the dataset
relevant_cols.remove("(Background Information, Position)")
relevant_cols.remove("(Background Information, Year of Birth)")

def radar_plot(player_names, player_data, reference_data, attribute_labels, reference_player_name):
    # Converting attribute labels to numpy array for manipulation
    labels = np.array(attribute_labels)
    num_vars = len(labels)

    # Computing angles for radar chart axes
    angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()

    # Creating a figure and a set of subplots with polar projection
    fig, ax = plt.subplots(figsize=(15, 15), subplot_kw=dict(polar=True))
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)

    # Setting ticks and labels for each angle (attribute)
    ax.set_xticks(angles)
    ax.set_xticklabels(attribute_labels)

    # Setting radial scale with logarithmic scaling
    ax.set_rscale('symlog')
    plt.yticks([-3, -2, -1, 0, 1, 2, 3], ["-3 SD", "-2 SD", "-1 SD", "Mean", "1 SD", "2 SD", "3 SD"], color="grey", size=7)
    plt.ylim(-3, 3)

    # Plotting radar charts for each player's data
    for i in range(len(player_names)):
        stats = player_data[i]
        if player_names[i] == reference_player_name:
            reference_stats = reference_data
            ax.plot(angles, reference_stats, linewidth=2, linestyle='solid', label=player_names[i])
        else:
            ax.plot(angles, stats, linewidth=1, linestyle='solid', label=player_names[i])

    # Adding a legend to the plot
    plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))

    plt.savefig(f'Exports/Reports/Similarity/{reference_player_name}/{reference_player_name} - Extended Radar Plot.jpg', bbox_inches='tight')

    # Displaying the radar plot
    return plt.show()

# Extracting player names and normalized data for plotting
player_names = [df.loc[player_index]['(Background Information, Full Name)'] for player_index, _ in top_similar_players]
player_data = [normalized_df.loc[player_index][relevant_cols] for player_index, _ in top_similar_players]

# Extracting reference player's index and data
reference_player_index = df[df['(Background Information, Full Name)'] == reference_player_name].index[0]
reference_data = normalized_df.loc[reference_player_index][relevant_cols]

# Inserting reference player's name and data to the beginning of lists
player_names.insert(0, reference_player_name)
player_data.insert(0, reference_data)

# Setting attribute labels
attribute_labels = relevant_cols

# Generating radar plot
radar_plot(player_names, player_data, reference_data, attribute_labels, reference_player_name)

## Export Comparison Data to CSV File

The Python code snippet provided above defines a function named `export_comparison_csv` aimed at exporting comparison data to a CSV (Comma-Separated Values) file. The function takes several parameters including `player_names`, `player_data`, `attribute_labels`, `reference_player_name`, and `df`, representing the names of players, their corresponding data, labels for attributes, the name of the reference player, and a DataFrame respectively. The function begins by initializing a list `all_attributes` which contains a comprehensive list of attributes categorized into various sections such as background information, playing time, performance, expected statistics, progression, and per 90 minutes metrics. These attributes serve as the column headers for the CSV file.

After that, the function extracts the original player data from the DataFrame `df` based on the provided player names. This is achieved by locating the indices of the players within the DataFrame and retrieving their respective data for the specified attributes. The extracted player data is then structured into a DataFrame named `comparison_data`, where each row corresponds to a player and each column corresponds to a specific attribute. Additionally, the player names are added as a new column to facilitate identification. Finally, the function exports the comparison data to a CSV file, using the reference player's name as part of the file name. The exported CSV file is stored in the directory "Exports/Reports/Similarity". Upon successful export, a message indicating the file path is displayed. To recap, this function provides a systematic approach to exporting structured comparison data, facilitating further analysis and visualization for all of the interested parties.

In [ ]:
import pandas as pd

# Function to format currency values
def format_currency(value):
    return f"{value:,.0f}€"

# Function to export comparison data to a CSV file
def export_comparison_csv(player_names, player_data, attribute_labels, reference_player_name, df):
    # List of all attributes in the desired order
    all_attributes = [
        "(Background Information, Squad)",
        "(Background Information, League)",
        "(Background Information, Position)",
        "(Background Information, Nation)",
        "(Background Information, Age)",
        "(Background Information, Year of Birth)",
        "(Playing Time, MP)",
        "(Playing Time, Starts)",
        "(Playing Time, Min)",
        "(Playing Time, 90s)",
        "(Performance, Gls)",
        "(Performance, Ast)",
        "(Performance, G+A)",
        "(Performance, G-PK)",
        "(Performance, PK)",
        "(Performance, PKatt)",
        "(Performance, CrdY)",
        "(Performance, CrdR)",
        "(Expected, xG)",
        "(Expected, npxG)",
        "(Expected, xAG)",
        "(Expected, npxG+xAG)",
        "(Progression, PrgC)",
        "(Progression, PrgP)",
        "(Progression, PrgR)",
        "(Per 90 Minutes, Gls)",
        "(Per 90 Minutes, Ast)",
        "(Per 90 Minutes, G+A)",
        "(Per 90 Minutes, G-PK)",
        "(Per 90 Minutes, G+A-PK)",
        "(Per 90 Minutes, xG)",
        "(Per 90 Minutes, xAG)",
        "(Per 90 Minutes, xG+xAG)",
        "(Per 90 Minutes, npxG)",
        "(Per 90 Minutes, npxG+xAG)",
        '(Market Value, Euro)',
        '(Peak Market Value, Euro)'
    ]
    
    # Converting columns to remove floating point
    df["(Background Information, Year of Birth)"] = df["(Background Information, Year of Birth)"].astype(int)
    df["(Playing Time, MP)"] = df["(Playing Time, MP)"].astype(int)
    df["(Playing Time, Starts)"] = df["(Playing Time, Starts)"].astype(int)
    df["(Playing Time, Min)"] = df["(Playing Time, Min)"].astype(int)
    df["(Performance, Gls)"] = df["(Performance, Gls)"].astype(int)
    df["(Performance, Ast)"] = df["(Performance, Ast)"].astype(int)
    df["(Performance, G+A)"] = df["(Performance, G+A)"].astype(int)
    df["(Performance, G-PK)"] = df["(Performance, G-PK)"].astype(int)
    df["(Performance, PK)"] = df["(Performance, PK)"].astype(int)
    df["(Performance, PKatt)"] = df["(Performance, PKatt)"].astype(int)
    df["(Performance, CrdY)"] = df["(Performance, CrdY)"].astype(int)
    df["(Performance, CrdR)"] = df["(Performance, CrdR)"].astype(int)
    df["(Progression, PrgC)"] = df["(Progression, PrgC)"].astype(int)
    df["(Progression, PrgP)"] = df["(Progression, PrgP)"].astype(int)
    df["(Progression, PrgR)"] = df["(Progression, PrgR)"].astype(int)
    
    # Format currency columns
    df['(Market Value, Euro)'] = df['(Market Value, Euro)'].map(format_currency)
    df['(Peak Market Value, Euro)'] = df['(Peak Market Value, Euro)'].map(format_currency)
    
    # Extracting original player data from the DataFrame df
    player_indices = [df[df['(Background Information, Full Name)'] == player_name].index[0] for player_name in player_names]
    player_data_original = [df.loc[player_index][all_attributes] for player_index in player_indices]

    # Creating DataFrame with player data
    comparison_data = pd.DataFrame(player_data_original, columns=all_attributes)
    
    # Creating a temporary list for player names with the suffix "(Reference)"
    modified_player_names = [f"{player_names[0]} (Reference)"] + player_names[1:]
    
    # Adding player names as a column
    comparison_data.insert(0, 'Player Name', modified_player_names)
    
    # Exporting to a CSV file
    file_name = f"Exports/Reports/Similarity/{reference_player_name}/{reference_player_name} - Comparison.csv"
    comparison_data.to_csv(file_name, index=False)
    print(f"Comparison data has been successfully exported to '{file_name}'.")

# Function call to export data
export_comparison_csv(player_names, player_data, attribute_labels, reference_player_name, df)

## In Conclusion

The Offside project was born with the idea of developing a solid, modular and efficient recommendation system, trying to offer personalized suggestions on football players based on the analysis and modeling of a constantly updated set of technical-tactical data. By exploiting a multidimensional approach based on the use of the expansion of a Cartesian plane and Euclidean distances, the project aimed to identify football players with the best similarity to a player taken as a reference.

Central to the project was the idea of building an algorithm from scratch that was capable of exploiting Euclidean distance calculations to quantify player similarities in an n-dimensional space. The resulting model was able to compute the required similarities, doing so with high computational efficiency, and minimizing the time investments required to use the model in practice. Not only this, but the model also presents great modularity: a professional in the analytical sector can easily modify its parameters to obtain a personalized and satisfactory result based on his needs.

Everything is completed by the visualization aspect, which provides intuitive representations that can be used not only by an analyst, but also by non-technical personnel, who can thus easily understand the insights deriving from the model. By mixing data-driven methodologies, a little creativity and machine learning, the project succeeded in its aim of building a framework capable of facilitating informed decision-making, optimizing player selection strategies in the dynamic world of football.